In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install keras
!pip install tensorflow

In [3]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Layer, Lambda

In [4]:
# load dataset
FF = pd.read_csv('C:/PERSONAL/ExcelR_Data-Science/Assignments/16_Neural_Networks/forestfires.csv')

In [5]:
#As dummy variables are already created, we will remove the month and alsoday columns
FF.drop(["month","day"],axis=1,inplace = True)

In [6]:
FF.columns

Index(['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area',
       'dayfri', 'daymon', 'daysat', 'daysun', 'daythu', 'daytue', 'daywed',
       'monthapr', 'monthaug', 'monthdec', 'monthfeb', 'monthjan', 'monthjul',
       'monthjun', 'monthmar', 'monthmay', 'monthnov', 'monthoct', 'monthsep',
       'size_category'],
      dtype='object')

In [7]:
FF["size_category"].value_counts()

small    378
large    139
Name: size_category, dtype: int64

In [8]:
FF.isnull().sum()

FFMC             0
DMC              0
DC               0
ISI              0
temp             0
RH               0
wind             0
rain             0
area             0
dayfri           0
daymon           0
daysat           0
daysun           0
daythu           0
daytue           0
daywed           0
monthapr         0
monthaug         0
monthdec         0
monthfeb         0
monthjan         0
monthjul         0
monthjun         0
monthmar         0
monthmay         0
monthnov         0
monthoct         0
monthsep         0
size_category    0
dtype: int64

In [9]:
FF.describe()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,...,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292,0.164410,...,0.017408,0.038685,0.003868,0.061896,0.032882,0.104449,0.003868,0.001934,0.029014,0.332689
std,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818,0.371006,...,0.130913,0.193029,0.062137,0.241199,0.178500,0.306138,0.062137,0.043980,0.168007,0.471632
min,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
##Considering small as 0 and large as 1
FF.loc[FF["size_category"]=='small','size_category']=0
FF.loc[FF["size_category"]=='large','size_category']=1
FF["size_category"].value_counts()

0    378
1    139
Name: size_category, dtype: int64

In [11]:
#Normalization
def norm_func(i):
     x = (i-i.min()) / (i.max() - i.min())
     return (x)

In [12]:
FF.columns

Index(['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area',
       'dayfri', 'daymon', 'daysat', 'daysun', 'daythu', 'daytue', 'daywed',
       'monthapr', 'monthaug', 'monthdec', 'monthfeb', 'monthjan', 'monthjul',
       'monthjun', 'monthmar', 'monthmay', 'monthnov', 'monthoct', 'monthsep',
       'size_category'],
      dtype='object')

In [13]:
FF.head()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,1,...,0,0,0,0,1,0,0,0,0,0
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,1,...,0,0,0,0,1,0,0,0,0,0
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0,...,0,0,0,0,1,0,0,0,0,0


In [14]:
predictors = FF.iloc[:,0:28]
target = FF.iloc[:,28]

In [15]:
predictors1 = norm_func(predictors)

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(predictors1,target, test_size=0.3,stratify = target)

In [17]:
def prep_model(hidden_dim):
    model = Sequential()
    for i in range(1,len(hidden_dim)-1):
        if (i==1):
            model.add(Dense(hidden_dim[i],input_dim=hidden_dim[0],activation="relu"))
        else:
            model.add(Dense(hidden_dim[i],activation="relu"))
    model.add(Dense(hidden_dim[-1],kernel_initializer="normal",activation="sigmoid"))
    model.compile(loss="binary_crossentropy",optimizer = "rmsprop",metrics = ["accuracy"])
    return model

In [18]:
first_model = prep_model([28,50,40,20,1])
first_model.fit(np.array(x_train).astype(np.float32),np.array(y_train).astype(np.float32),epochs=500)
pred_train = first_model.predict(np.array(x_train))

Epoch 1/500
12/12 [==============================] - 1s 2ms/step - loss: 0.6854 - accuracy: 0.6316
Epoch 2/500
12/12 [==============================] - 0s 2ms/step - loss: 0.6540 - accuracy: 0.7313
Epoch 3/500
12/12 [==============================] - 0s 2ms/step - loss: 0.6166 - accuracy: 0.7313
Epoch 4/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5926 - accuracy: 0.7313
Epoch 5/500
12/12 [==============================] - 0s 3ms/step - loss: 0.5864 - accuracy: 0.7313
Epoch 6/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5829 - accuracy: 0.7313
Epoch 7/500
12/12 [==============================] - ETA: 0s - loss: 0.5925 - accuracy: 0.71 - 0s 2ms/step - loss: 0.5833 - accuracy: 0.7313
Epoch 8/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5802 - accuracy: 0.7313
Epoch 9/500
12/12 [==============================] - 0s 2ms/step - loss: 0.5797 - accuracy: 0.7313
Epoch 10/500
12/12 [==============================] - 0s 2ms/step -

12/12 [==============================] - 0s 2ms/step - loss: 0.3447 - accuracy: 0.8560
Epoch 83/500
12/12 [==============================] - 0s 2ms/step - loss: 0.3396 - accuracy: 0.8615
Epoch 84/500
12/12 [==============================] - 0s 2ms/step - loss: 0.3319 - accuracy: 0.8587
Epoch 85/500
12/12 [==============================] - 0s 2ms/step - loss: 0.3325 - accuracy: 0.8504
Epoch 86/500
12/12 [==============================] - 0s 2ms/step - loss: 0.3322 - accuracy: 0.8781
Epoch 87/500
12/12 [==============================] - 0s 2ms/step - loss: 0.3203 - accuracy: 0.8532
Epoch 88/500
12/12 [==============================] - 0s 2ms/step - loss: 0.3168 - accuracy: 0.8698
Epoch 89/500
12/12 [==============================] - 0s 2ms/step - loss: 0.3145 - accuracy: 0.8643
Epoch 90/500
12/12 [==============================] - 0s 2ms/step - loss: 0.3007 - accuracy: 0.8753
Epoch 91/500
12/12 [==============================] - 0s 2ms/step - loss: 0.3125 - accuracy: 0.8753
Epoch 92/500


12/12 [==============================] - 0s 2ms/step - loss: 0.1290 - accuracy: 0.9529
Epoch 164/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1102 - accuracy: 0.9640
Epoch 165/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1174 - accuracy: 0.9557
Epoch 166/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1471 - accuracy: 0.9446
Epoch 167/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1031 - accuracy: 0.9612
Epoch 168/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1154 - accuracy: 0.9557
Epoch 169/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1227 - accuracy: 0.9474
Epoch 170/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1454 - accuracy: 0.9363
Epoch 171/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0968 - accuracy: 0.9612
Epoch 172/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1327 - accuracy: 0.9446
Epoc

12/12 [==============================] - 0s 2ms/step - loss: 0.0693 - accuracy: 0.9668
Epoch 245/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0385 - accuracy: 0.9917
Epoch 246/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0867 - accuracy: 0.9668
Epoch 247/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0694 - accuracy: 0.9723
Epoch 248/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0428 - accuracy: 0.9889
Epoch 249/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0633 - accuracy: 0.9751
Epoch 250/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0653 - accuracy: 0.9668
Epoch 251/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0539 - accuracy: 0.9723
Epoch 252/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0654 - accuracy: 0.9695
Epoch 253/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0516 - accuracy: 0.9834
Epoc

12/12 [==============================] - 0s 2ms/step - loss: 0.0931 - accuracy: 0.9529
Epoch 326/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0144 - accuracy: 1.0000
Epoch 327/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0373 - accuracy: 0.9861
Epoch 328/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0197 - accuracy: 0.9972
Epoch 329/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0225 - accuracy: 0.9917
Epoch 330/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0402 - accuracy: 0.9834
Epoch 331/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0526 - accuracy: 0.9778
Epoch 332/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0283 - accuracy: 0.9889
Epoch 333/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0165 - accuracy: 1.0000
Epoch 334/500
12/12 [==============================] - ETA: 0s - loss: 0.0171 - accuracy: 1.00 - 0s 2ms/s

12/12 [==============================] - 0s 3ms/step - loss: 0.0085 - accuracy: 1.0000
Epoch 406/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0294 - accuracy: 0.9917
Epoch 407/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0534 - accuracy: 0.9778
Epoch 408/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 409/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 410/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0378 - accuracy: 0.9889
Epoch 411/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0242 - accuracy: 0.9889
Epoch 412/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0084 - accuracy: 0.9972
Epoch 413/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0735 - accuracy: 0.9695
Epoch 414/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0051 - accuracy: 1.0000
Epoc

12/12 [==============================] - 0s 3ms/step - loss: 0.0516 - accuracy: 0.9806
Epoch 485/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 486/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 487/500
12/12 [==============================] - 0s 3ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 488/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0734 - accuracy: 0.9751
Epoch 489/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 490/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 491/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 492/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0184 - accuracy: 0.9945
Epoch 493/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0442 - accuracy: 0.9861
Epoc

In [19]:
#Converting the predicted values to series 
pred_train = pd.Series([i[0] for i in pred_train])

In [20]:
size = ["small","large"]
pred_train_class = pd.Series(["small"]*361)
pred_train_class[[i>0.5 for i in pred_train]]= "large"

In [21]:
train = pd.concat([x_train,y_train],axis=1)
train["size_category"].value_counts()

0    264
1     97
Name: size_category, dtype: int64

In [22]:
#For training data
from sklearn.metrics import confusion_matrix
train["original_class"] = "small"
train.loc[train["size_category"]==1,"original_class"] = "large"
train.original_class.value_counts()

small    264
large     97
Name: original_class, dtype: int64

In [23]:
confusion_matrix(pred_train_class,train["original_class"])

array([[ 97,   0],
       [  0, 264]], dtype=int64)

In [24]:
np.mean(pred_train_class==pd.Series(train["original_class"]).reset_index(drop=True))

1.0

In [25]:
pd.crosstab(pred_train_class,pd.Series(train["original_class"]).reset_index(drop=True))

original_class,large,small
row_0,,
large,97,0
small,0,264


In [26]:
#For test data
pred_test = first_model.predict(np.array(x_test))
pred_test = pd.Series([i[0] for i in pred_test])
pred_test_class = pd.Series(["small"]*156)
pred_test_class[[i>0.5 for i in pred_test]] = "large"
test =pd.concat([x_test,y_test],axis=1)
test["original_class"]="small"
test.loc[test["size_category"]==1,"original_class"] = "large"

In [27]:
test["original_class"].value_counts()

small    114
large     42
Name: original_class, dtype: int64

In [28]:
np.mean(pred_test_class==pd.Series(test["original_class"]).reset_index(drop=True))

0.9102564102564102

In [29]:
confusion_matrix(pred_test_class,test["original_class"])

array([[ 28,   0],
       [ 14, 114]], dtype=int64)

In [30]:
pd.crosstab(pred_test_class,pd.Series(test["original_class"]).reset_index(drop=True))

original_class,large,small
row_0,,
large,28,0
small,14,114
